# ABA: Recitation 5

### Spring 2024

In [ ]:
import os
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import poisson
import seaborn as sns

- Prediction in poisson
- NBD
- Zero Inflated Note

# Household Needs

International agencies use household size to determine the magnitude of the household needs. I am attaching a dataset on a survey from Philippines which contains information on:

- the region,
- the number of people living in the household,
- age of the head of the household,
- number of children below age 5 and 
- type of roof used.

**First, we aggregate the data by 5-year age cohort and find the mean and the variance of the number of people in the household in each cohort. This should help you understand whether your assumption of mean = variance is a reasonable one.**

In [ ]:
#read data
df_q1 = pd.read_excel('household_data.xlsx',sheet_name='household')
df_q1.rename(columns={'total_households':'total_household_size'},inplace=True)
cols_to_keep = ['location','age','total_household_size','roof']
df_q1 = df_q1[cols_to_keep].reset_index(drop=True)

In [ ]:
df_q1.describe()

In [ ]:
df_q1.groupby('location').size()

In [ ]:
#plot distribution of counts
sns.histplot(data=df_q1,x='total_household_size')
plt.show()

In [ ]:
# define the minimum and maximum age
min_age = df_q1.age.min() #18
max_age = df_q1.age.max() #98

# define the size of each bin
bin_size = 5

# generate the bin edges
bin_edges = range(min_age-1,(max_age+bin_size),bin_size)

# create the age groups by applying the pd.cut function
df_q1['age_group'] = pd.cut(df_q1['age'], bins=bin_edges)

# print the resulting dataframe
df_q1

In [ ]:
#mean and variance for each cohort
df_agg = df_q1.groupby('age_group')['total_household_size'].agg(['count','mean','std']).reset_index()
df_agg

mean and variance are close in some cohorts, but not in others

**In GLM, for Poisson model, your regress the mean of the counts log(y) with covariates in a linear fashion. 
We will plot log(mean of the size of household) with age. To do this we rely on our earlier calculation where you aggregated the data.**

In [ ]:
#aggregate by group and compute the log of the mean
df_agg['age_group_bound'] = df_agg.age_group.apply(lambda x: x.left)
df_agg['mean_log'] = np.log(df_agg['mean'])
df_agg['age_group_str'] = df_agg.age_group.astype('str')

In [ ]:
#plot age groups against mean size of household
sns.scatterplot(data=df_agg,x='age_group_bound',y='mean_log')
plt.title("Mean count vs Age")
plt.xlabel('Age')
plt.ylabel('Mean Count')
plt.show()

In [ ]:
# #We can also do it with the whole data
# df_q1['age_group_bound'] = df_q1.age_group.apply(lambda x: x.left)
# df_q1['log_hh_size'] = np.log(df_q1['total_household_size'])
# df_q1['age_group_str'] = df_q1.age_group.astype('str')

# sns.scatterplot(data=df_q1,x='age_group_bound',y='log_hh_size')
# plt.title("Log Household Size vs Age")
# plt.xlabel('Age')
# plt.ylabel('Log Household Size')
# plt.show()

The linear assumption between age group and log($\lambda$) does not seem to hold. The log($\lambda$) seems to follow a non linear pattern. Increasing with age up until the mid 40s, and then starts decreasing. 

# Poisson

**Now we estimate the Poisson model. First we include age and location dummies as covariates. Then, we will add age$^2$ to deal with the non-linearity**

In [ ]:
#create age^2

In [ ]:
df_q1['age2'] = np.power(df_q1.age,2)

In [ ]:
df_q1.groupby('location').size()

We are estimating the mean size of the househole as a function of the covariates:

\begin{equation}
\log(\lambda_i) = \beta_0 + \beta_1 \times \text{DavaoRegion} + \beta_2 \times \text{IlocosRegion} + \beta_3 \times \text{MetroManila} + \beta_4 \times \text{Visayas} + \beta_5 \times \text{age}
\end{equation}

Where: 

\begin{align*}
\lambda & \text{ is the expected size of the household}, \\
\beta_0 & \text{ is the intercept}, \\
\beta_1 - \beta_4 & \text{ are the coefficients for Location dummies}, \\
\beta_5 & \text{ is the coefficient for the variable "Age"}\\
\end{align*}


Note on estimation:

Recall that the paramaters are found by maximizing the likelihood function.

For the entire dataset:

$$L(\mathbf{y}; \boldsymbol{\lambda}) = \prod_{i=1}^{n} \frac{e^{-\lambda_i} \lambda_i^{y_i}}{y_i!} $$

the overall log-likelihood for the entire dataset is the sum of the individual log-likelihoods for each observation:

$$\log L(\mathbf{y}; \boldsymbol{\lambda_i}) = \sum_{i=1}^{n} (-\lambda_i + y_i \log(\lambda_i) - \log(y_i!))$$

Where:
- $\mathbf{y}$ is the vector of observed counts.
- $\boldsymbol{\lambda}_{i}$ is the vector of expected size of household modeled as a function of the predictors.


In [ ]:
#Create the training and test data sets. 
np.random.seed(42)
mask = np.random.rand(len(df_q1)) < 0.8
df_train = df_q1[mask]
df_test = df_q1[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

In [ ]:
#model 1
expr1 = 'total_household_size ~ age + location'
print("Expresion: %s" % expr1)

y_train, X_train = dmatrices(expr1, df_train, return_type='dataframe')
y_test, X_test1 = dmatrices(expr1, df_test, return_type='dataframe')

#Using the statsmodels GLM class, we first train the Poisson regression model on the training data set.
poisson_training_results1 = sm.GLM(endog=y_train
                                    ,exog=X_train
                                    ,family=sm.families.Poisson()).fit()
print(poisson_training_results1.summary())

Include $Age^2$ to deal with non-linearity of the data:
    
\begin{equation}
\log(\lambda_i) = \beta_0 + \beta_1 \times \text{DavaoRegion} + \beta_2 \times \text{IlocosRegion} + \beta_3 \times \text{MetroManila} + \beta_4 \times \text{Visayas} + \beta_5 \times \text{age} + \beta_5 \times \text{age}^2
\end{equation}


In [ ]:
#model 2
expr2 = 'total_household_size ~ age + location + age2'
print("Expresion: %s" % expr2)
y_train, X_train = dmatrices(expr2, df_train, return_type='dataframe')
y_test, X_test2 = dmatrices(expr2, df_test, return_type='dataframe')


#Using the statsmodels GLM class, we first train the Poisson regression model on the training data set.
poisson_training_results_2 = sm.GLM(endog=y_train
                                    ,exog= X_train
                                    ,family=sm.families.Poisson()).fit()
print(poisson_training_results_2.summary())

- **Intercept (-0.5043)**: The intercept represents the log of the expected count of total household size when all the predictor variables are 0. Since age and age2 can't really be 0 in this context, this value is more theoretical. For categorical variables, it's the baseline against which other categories are compared. Here, the baseline is the log of the expected total household size for the reference location (not explicitly mentioned but implied to be the one not listed among the location categories) and when age is 0. 

- **location[T.DavaoRegion] (-0.0012)**: Being in the Davao Region, compared to the baseline location, changes the log of the expected total household size by -0.0012. This effect is very small and not statistically significant (p-value: 0.984), suggesting that being in Davao Region does not significantly affect the total household size compared to the baseline.

- **location[T.IlocosRegion] (0.0204)**: Being in the Ilocos Region, compared to the baseline location, changes the log of the expected total household size by 0.0204. This is also a small and not statistically significant effect (p-value: 0.735), indicating no significant difference in total household size for Ilocos Region compared to the baseline.

- **location[T.MetroManila] (0.0694)**: Being in Metro Manila, compared to the baseline location, changes the log of the expected total household size by 0.0694. Though this coefficient is larger than for other regions, it is not statistically significant (p-value: 0.183), suggesting that being in Metro Manila does not significantly alter the expected total household size compared to the baseline.

- **location[T.Visayas] (0.1063)**: Being in the Visayas, compared to the baseline location, increases the log of the expected total household size by 0.1063. This coefficient **is statistically significant** (p-value: 0.021), indicating a significant effect on the total household size for those in the Visayas compared to the baseline.

- **age (0.0753)**: Each additional year of age increases the log of the expected total household size by 0.0753. This is a statistically significant effect (p-value < 0.001), meaning as age increases, the total household size is expected to increase.

- **age2 (-0.0007)**: The squared term of age indicates a non-linear relationship between age and the log of the expected total household size. Each additional year squared decreases the log of the expected total household size by 0.0007. This effect is statistically significant (p-value < 0.001), suggesting that the increase in total household size with age diminishes as age increases.


The model with only age and location as covariate shows a significant negative coefficient, suggesting a negative association between age and houshold. An estimate -0.0047 suggest that when age goes up by a unit, the household size goes down by exp(-0.0047) or about half a percent.

Once we add age$^2$, we observe that both age and age$^2$ have a significant coefficient. Now, age has a positive coefficient, and age$^2$ has a negative coefficient. This suggests that as people get older, the positive effect of age decreases (as the plot we did earlier suggests).

In [ ]:
#Example effect of age:
def effect_age(age):
    return 0.0753*age -0.0007*np.power(age,2)

age_range = np.arange(18, 98+1)

sns.lineplot(x=age_range,y=effect_age(age_range))
sns.lineplot(x=age_range, y=[0]*len(age_range))

# NBD

What if we expect heterogeneity?

We can allow $\lambda$ to be heterogeneous. In particular, it will follow a Gamma distribution. The likelihood (probability of observing the data) is then:


$$f_m(y) = f\left(y|\lambda\right) f(\lambda)$$

where $f\left(y|\lambda\right)$ is the Poisson distribution (conditioned on $\lambda$), and $f(\lambda)$ is the distribution of $\lambda$ (assumed to be gamma).

It turns out that a mixture of Poisson $f(y|\lambda)$ with gamma $f(\lambda)$ leads to the widely used **negative binomial distribution (NBD)**. The probability distribution for the NBD is given by:
$$P(y; p, r) = \frac{(y + r - 1)!}{y! (r - 1)!} p^r (1 - p)^y$$
where:
- $y$ is the number of failures,
- $r$ is the number of successes,
- and $p$ is the probability of success.


### Estimating the model in Python

The expected value of counts as a function of the predictors:

$$log(\lambda) = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p $$

The variance of the Negative Binomial distribution in terms of $\lambda$ (i.e  $E(Y|X)$) and the dispersion parameter $\alpha$:

$$\text{Var}(Y|X) = \lambda + \alpha \lambda^2$$

In this formula:

- The term $\lambda$ is the mean (note it is also the variance in the case of a Poisson distribution).
- The term $\alpha \lambda^2$ accounts for the extra variability (overdispersion) beyond what's expected in a Poisson distribution.


***Note that the how $\alpha = 0$ leads to Poisson***


We estimate the parameters by doing Maximum Likelihood Estimation.

Given the Negative Binomial parameterization used in Statsmodel:

$$f(y; \lambda, \alpha) = \frac{\Gamma(y + \frac{1}{\alpha})}{y! \Gamma(\frac{1}{\alpha})} \left( \frac{1}{1 + \alpha \lambda} \right)^{\frac{1}{\alpha}} \left( \frac{\alpha \lambda}{1 + \alpha \lambda} \right)^y $$

Where:
- $ y $ is the observed count.
- $ \lambda $ is the mean related to the predictors as: $ \log(\lambda) = X^T\beta $
- $ \alpha $ is the dispersion parameter.

The likelihood for observed data $( Y_1, Y_2, ..., Y_n)$ with predictors $ X_1, X_2, ..., X_n $ is:

$$ L(\beta, \alpha | Y, X) = \prod_{i=1}^{n} \frac{\Gamma(Y_i + \frac{1}{\alpha})}{Y_i! \Gamma(\frac{1}{\alpha})} \left( \frac{1}{1 + \alpha \exp(X_i^T \beta)} \right)^{\frac{1}{\alpha}} \left( \frac{\alpha \exp(X_i^T \beta)}{1 + \alpha \exp(X_i^T \beta)} \right)^{Y_i} $$

Source: https://www.statsmodels.org/stable/generated/statsmodels.genmod.families.family.NegativeBinomial.html


**We believe that data is heterogeneous and estimate the Negative Binomial Model with the age and location as covariates.**

In [ ]:
#create expression and dmatrices
expr_nbd = 'total_household_size ~ age + location'
y_train, X_train = dmatrices(expr_nbd, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr_nbd, df_test, return_type='dataframe')

In [ ]:
#fit NBD model using Negative binomial
NBD_training_results = sm.NegativeBinomial(endog=y_train
                                           ,exog=X_train).fit()
print(NBD_training_results.summary())

- **Intercept (1.4920)**: The intercept represents the log of the expected count of the total household size when all predictor variables are at their reference levels (CentralLuzon). 

- **Location Coefficients**:
  - **Davao Region (0.0117)**: Being in the Davao Region changes the log of the expected total household size by 0.0117 compared to the baseline location. However, this effect is not statistically significant (p-value: 0.867), suggesting that being in Davao Region does not significantly alter the expected total household size compared to the baseline.
  - **Ilocos Region (0.0075)**: Being in the Ilocos Region changes the log of the expected total household size by 0.0075 compared to the baseline location. This effect is also not statistically significant (p-value: 0.916), indicating no significant difference in total household size for Ilocos Region compared to the baseline.
  - **Metro Manila (0.0936)**: Being in Metro Manila changes the log of the expected total household size by 0.0936 compared to the baseline location. The effect is not statistically significant (p-value: 0.130), suggesting that being in Metro Manila does not significantly alter the expected total household size compared to the baseline.
  - **Visayas (0.1248)**: Being in the Visayas increases the log of the expected total household size by 0.1248 compared to the baseline location. This coefficient **is statistically significant** (p-value: 0.023), indicating a significant effect on the total household size for those in the Visayas compared to the baseline.

- **Age (-0.0049)**: Each additional year of age decreases the log of the expected total household size by -0.0049. This is a statistically significant effect (p-value < 0.000), meaning as age increases, the total household size is expected to decrease slightly.

- **Alpha (0.1148)**: The alpha parameter in a Negative Binomial Regression model measures the degree of over-dispersion in the count data. A value of 0 would indicate that the data is not over-dispersed (i.e., a Poisson distribution is appropriate), while positive values indicate the presence of over-dispersion. Here, the statistically significant alpha (p-value < 0.000) suggests that there is over-dispersion in the total household size data, and the Negative Binomial model is a better fit than a Poisson model. In Poisson, mean=variance, and in the NBD model, variance = mean + alpha*mean2. The value of 0.1148 indicates the extent of over-dispersion, with higher values signifying more over-dispersion. 

**When you predict the outcome, what do you predict? We will plot the histogram of actual and predicted number of households. Why does the prediction doesn't match the data well?**

The predict functions estimates the expected value of people in the household given the covariates:
$$\lambda_i = exp(\beta_0 + \beta_1 \times \text{DavaoRegion} + \beta_2 \times \text{IlocosRegion} + \beta_3 \times \text{MetroManila} + \beta_4 \times \text{Visayas} + \beta_5 \times \text{age}) $$


In [ ]:
np.exp(1.491966+0.093585+75*-0.004935)

In [ ]:
#NBD PREDICTIONS
nbd_hh_predicted_mean = NBD_training_results.predict(X_test)
y_test['nbd_hh_predicted_mean'] = nbd_hh_predicted_mean
y_test

In [ ]:
# Set up the Matplotlib figure
plt.figure(figsize=(8, 6))
# Plot the 'nbd_hh_predicted_mean' histogram
sns.histplot(data=y_test, x='nbd_hh_predicted_mean', color='red', binwidth=1, label='Predicted Means', alpha=0.5)

# Plot the 'total_household_sizes' histogram
sns.histplot(data=y_test, x='total_household_size', color='blue', binwidth=1, label='Number of people in the HH (actual)', alpha=0.5)

# Show the legend
plt.legend()

# Display the plot
plt.show()

Since the covariates do not explain the variation, individual level predictions collpase towards the mean. Hence, the predictions using the mean do not match the actual data well. Using the random number generators we get better predictions. We didn't discuss this in class, but you can use this as reference:

We'll plot the counts using the random number generator

In [ ]:
rng = np.random.default_rng()

In [ ]:
## just simulate one sample for each observation 
## use this as the prediction 
params = NBD_training_results.params
gamma=np.exp(params[:-1]@ X_test.T) #exp(XB)
alpha=np.exp(params[-1]) #alpha
rng = np.random.default_rng()
y_test['nbd_hh_random_draw'] = rng.negative_binomial(n=gamma,p=alpha/(1+alpha)) #n=exp(XB), p=(alpha/(1+alpha))

In [ ]:
# Set up the Matplotlib figure
plt.figure(figsize=(8, 6))
# Plot the 'nbd_hh_random_draw' histogram
sns.histplot(data=y_test, x='nbd_hh_random_draw', color='red', binwidth=0.5, label='NBD Draws', alpha=0.5)

# Plot the 'total_household_sizes' histogram
sns.histplot(data=y_test, x='total_household_size', color='blue', binwidth=0.5, label='Number of people in the HH (actual)', alpha=0.5)

# Show the legend
plt.legend()

# Display the plot
plt.show()

# Brief Note on Zero Inflated Poisson

The zero inflated model allows us to model data that has too many zero counts. To accommodate excessive zeros, we have to incorporate a structure which can predict probability of zero incidence. Thus, a Zero inflated Poisson model has two parts:

- A PMF P(y_i=0) which is used to calculate the probability of observing a zero count.
- A second PMF P(y_i=k) which is used to calculate the probability of observing k events, given that k > 0.

We can express it as:

$${\displaystyle \Pr(Y=0)=\pi +(1-\pi )e^{-\lambda }}$$

$${\displaystyle \Pr(Y=y_{i})=(1-\pi ){\frac {\lambda ^{y_{i}}e^{-\lambda }}{y_{i}!}},\qquad y_{i}=1,2,3,...}$$

where the outcome variable 
$y_{i}$ has any non-negative integer value, $\lambda$  is the expected Poisson count for the  ith individual; $\pi$  is the probability of extra zeros. (see https://en.wikipedia.org/wiki/Zero-inflated_model#Discrete_pseudo_compound_Poisson_model)

### Model

For the **Zero-Inflated Poisson (ZIP) model**:

1. **Logistic Component (Inflation model)**:
Given the probability:
$$\pi_i = \frac{e^{X_i^T \beta_{infl}}} {1 + e^{X_i^T \beta_{infl}}}$$

The log odds of an observation being in the "always-zero" group is:
$$\log\left(\frac{\pi_i}{1-\pi_i}\right) = X_i^T \beta_{infl}$$
Where:
- $ \log $ represents the natural logarithm.
- $ \pi_i $ is the probability that the i-th observation is always zero.
- $ X_i $ is the vector of predictors for the i-th observation.
- $ \beta_{infl} $ are the coefficients for the logistic regression component, which you are estimating.

2. **Poisson Component**:
The mean $ \lambda$ of the Poisson distribution (for observations not in the "always-zero" group) is modeled as:
$$ \log(\lambda_i) = X_i^T \beta_{pois} $$
Where:
- $\beta_{pois}$ are the coefficients of the Poisson regression component.


### Predictions using the Zero-Inflated Poisson model:

1. **Conditional Expectation**: The expected count, taking both zero inflation and the count distribution into account.
    ```python
    expected_value = zip_training_results.predict(exog=X_test, exog_infl=X_test, which='mean')
    ```

2. **Linear Predictor**: The linear combination of predictors before applying any link or count function.
    ```python
    linear_predictor = zip_training_results.predict(exog=X_test, exog_infl=X_test, which='linear')
    ```

3. **Estimated Variance**: The variance of the response variable as implied by the model.
    ```python
        Didn't work
    ```

4. **Mean of Main Model**: The expected count only from the main count model, ignoring zero inflation.
    ```python
    mean_main = zip_training_results.predict(exog=X_test, which='mean-main')
    ```

5. **Probability of Main Model**: The probability that an observation comes from the main count model. The complementary probability represents zero inflation.
    ```python
    prob_main = zip_training_results.predict(exog=X_test, exog_infl=X_test, which='prob-main')
    ```

6. **Expected Value for Non-zero Counts**: The expected count given that it is not zero.
    ```python
    mean_nonzero = zip_training_results.predict(exog=X_test, which='mean-nonzero')
    ```

7. **Probability of Zero Count**: The probability that the count is zero. 
    ```python
    prob_zero = zip_training_results.predict(exog=X_test, exog_infl=X_test, which='prob-zero')
    ```

8. **Probabilities for Each Count**: The probabilities for a range of count values.
    ```python
    Didn't work!
    ```

